# TREC 2018 Precision Medicine

In [1]:
import json
from json2html import *
from IPython.display import HTML
import eval

In [2]:
with open('config.json', 'r') as f:
    config = json.load(f)
#HTML(json2html.convert(json = config))

Load TOPICS and GOLD STANDARD from last year (30 topics)

In [3]:
topics_df = eval.get_topics('./topics/topics2017.xml')

In [4]:
#topics_df

In [5]:
run_df = eval.run(topics_df)

RUN: FIXME {'disease_tie_breaker': 0.5, 'gene_tie_breaker': 0.5, 'gene_boost': 1, 'disease_boost': 1.5, 'query_template': 'baseline.json'}


In [6]:
#run_df

In [7]:
qrels = eval.get_qrels('./gold-standard/sorted.abstracts.2017.qrels')

In [8]:
results, aggregated = eval.evaluate(qrels, run_df)

In [9]:
aggregated

{'P_10': 0.48, 'Rprec': 0.2847, 'ndcg': 0.5338}

In [10]:
results['1']['ndcg']

0.6888840985776884

In [11]:
results['3']['ndcg']

0.6298053155348644